In [1]:
import sys
sys.path.append("/home/jovyan/work/Architectures_for_Big_Data/")
import pyspark
sc = pyspark.SparkContext("local[3]")

In [27]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(intGenerator(),min=1,max=500,keyName="referenceId")
dataset.addGenerator(intGenerator(),min=1,max=5000,keyName="itemId")
dataset.addGenerator(dateGenerator(),min=1,max=10,keyName="ts")

activities = sc.parallelize(dataset.generateDataset(100000)).persist()

dataset2 = datasetGenerator()
dataset2.addGenerator(intGenerator(),min=1,max=500,keyName="referenceId")
dataset2.addGenerator(randomFromListGenerator(),list=['P&P103','P&P104','P&P105'],keyName="machineId")
dataset2.addGenerator(randomFromListGenerator(),list=['SMT24','SMT25','SMT26','SMT27'],keyName="lineId")
dataset2.addGenerator(randomFromListGenerator(),list=['In Progress', 'Idle','Done'],keyName="status")

lines = sc.parallelize(dataset2.generateDataset(1000)).persist()

In [28]:
activities = activities.map(lambda x: (x.get('referenceId'),(x.get('itemId'), x.get('ts'))))
activities.first()

(351, (886, datetime.datetime(2014, 1, 18, 2, 22, 9)))

In [29]:
# just to get only unique values
dist = lines.map(lambda x: ((x.get('machineId'), x.get('lineId')), x)).reduceByKey(lambda a,b: a)\
    .map(lambda x: x[1])\
    .filter(lambda x: x.get('lineId') == 'SMT24' or x.get('lineId') == 'SMT25')\
    .filter(lambda x: x.get('status') != 'Done')\
    .map(lambda x: (x.get('referenceId'),(x.get('lineId'))))
dist.first()

(164, 'SMT24')

In [30]:
# count the number of pieces done per line
# (lineId, amountOfPieces)

afterjoin = dist.join(activities).map(lambda x: (x[1][0], x[1][1][0], x[1][1][1]))
afterjoin.first()

('SMT25', 920, datetime.datetime(2017, 11, 29, 11, 20, 22))

In [31]:
perline = afterjoin.map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a+b)
perline.top(5, lambda x: x[1])

[('SMT25', 604), ('SMT24', 584)]

In [38]:
perday = afterjoin.map(lambda x: (x[2].weekday(),1)).reduceByKey(lambda a,b: a+b)
perday.collect()

[(0, 195), (6, 176), (1, 175), (2, 160), (3, 151), (4, 184), (5, 147)]

In [39]:
perlineday = afterjoin.map(lambda x: ((x[0], x[2].weekday()), 1)).reduceByKey(lambda a,b: a+b)
perlineday.collect()

[(('SMT25', 2), 86),
 (('SMT24', 2), 74),
 (('SMT25', 5), 82),
 (('SMT24', 5), 65),
 (('SMT25', 4), 94),
 (('SMT24', 6), 76),
 (('SMT24', 0), 100),
 (('SMT25', 3), 72),
 (('SMT25', 1), 75),
 (('SMT25', 0), 95),
 (('SMT25', 6), 100),
 (('SMT24', 4), 90),
 (('SMT24', 3), 79),
 (('SMT24', 1), 100)]